In [9]:
%pip install google-generativeai

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 155 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 28.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.3 MB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 426 kB 40.5 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 418 kB 26.0 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 4.4 MB/s  eta 0:00:01
     |████████████████████████████████| 91 kB 17.7 MB/s eta 0:00:01
You should consider upgradi

In [2]:
%pip install google-cloud-vision

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 527 kB 4.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
from google.cloud import vision
import time

# 1. Setup service account key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/gracelin/Desktop/Synced Air/fall 25/4_unstructured/assignment/assignment_3/gen-lang-client-0003990277-c8504c9e2ebf.json"

# 2. Initialize the Vision client
client = vision.ImageAnnotatorClient()

# 3. Function to label one image
def label_image(image_url, idx, total):
    if not isinstance(image_url, str) or image_url.strip() == "" or image_url == "nan":
        print(f"\n⚠️ Skipped {idx+1}/{total} (no image URL)")
        return "No image URL"

    try:
        image = vision.Image()
        image.source.image_uri = image_url

        # Main label detection
        response = client.label_detection(image=image)
        labels = [label.description for label in response.label_annotations]

        # If no labels (e.g., just text banners), fallback to OCR
        if not labels:
            text_response = client.text_detection(image=image)
            texts = [t.description for t in text_response.text_annotations]
            if texts:
                labels = ["TEXT: " + texts[0][:100]]

        result = ", ".join(labels) if labels else "No labels"

    except Exception as e:
        result = f"Error: {e}"

    # Print progress + preview
    print(f"\n✅ Processed {idx+1}/{total}")
    print(f"   URL: {image_url[:80]}...")
    print(f"   Labels: {result}\n")

    return result

# 4. Load campaigns
df = pd.read_csv("campaigns.csv", quoting=1)

# 5. Loop with progress tracking + output preview
total = len(df)
labels = []
for i, url in enumerate(df["cover_image"]):
    label = label_image(url, i, total)
    labels.append(label)
    time.sleep(0.2)  # avoid quota/rate-limit

df["image_labels"] = labels


E0000 00:00:1759391952.971503 5414824 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



✅ Processed 1/1000
   URL: https://images.gofundme.com/vfQ_a6e1pE7GZrlsdMgruTKPhPc=/1200x900/https://d2g8ig...
   Labels: Child, Hospital, Patient, Health Care, Medical equipment, Medical, Medical procedure, Infant, Toddler, Hospital bed


✅ Processed 2/1000
   URL: https://images.gofundme.com/ybPT_3fBXkCoE-5h3PPcrxLfvW0=/1200x900/https://d2g8ig...
   Labels: Mountainous landforms, Mountain, People in nature, Vacation, Recreation, Geology, Happiness, Leisure, Travel, Mountain range


✅ Processed 3/1000
   URL: https://images.gofundme.com/AMU6oV2WJKrYi40EftdY3LdF_Q8=/1200x900/https://d2g8ig...
   Labels: T-shirt, Arm, Wrist, Chair, Hearing, Active Shirt, Watch, Student


✅ Processed 4/1000
   URL: https://images.gofundme.com/DLe9STiNA-dP-2_kuAZwJc-y914=/1200x900/https://d2g8ig...
   Labels: Smile, Cheek, Happiness, Eyebrow, Facial hair, Lips, Forehead, Beard, Jaw, Mouth


✅ Processed 5/1000
   URL: https://images.gofundme.com/j256Flpa_mxzBPBEDlWHF4z_Vo8=/1200x900/https://d2g8ig...
   L

In [26]:
df.to_csv("campaigns_with_labels.csv", index=False, quoting=1)